#### EX 6.3 Vectors

1. Embed
   - 사람이 읽는 텍스트를 컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업
   - OpenAI에서는 최대 1,000차원의 Vector 값을 제공하지만 우리는 3차원만 사용해서 embed 한다
   - 자세한 설명은 아래의 영상 url 참조  
     https://nomadcoders.co/fullstack-gpt/lectures/4575

- YouTube 참고
  https://www.youtube.com/watch?v=2eWuYf-aZE4


#### Ex 6.4 Vector Store

2. Embeding (embed 실행시 비용발생)
   - Splitter로 분할한 문서를 OpenAI에서 제공하는 embeding 모델을 적용하여 문서를 embed 하자
   - Embeding 한 문서를 로컬 vector store에 저장해서 재활용하도록 한다. (비용을 아끼자.)
   - 여기서는 오픈소스인 Chroma를 사용한다.
   - Chroma에 분할한 문서와 openAI embeding model을 전달하자
     - OpenAI의 embeding은 text-embedding-ada-002 model을 사용한다.(가성비 짱)

- Vector Store 중 유명한 것은 pinecone이 있다


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

# separator를 사용하면 특정 문자열을 지정하여 문장을 분할할수 있다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,  # 최대 글자 개수
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# .cache에 저장된 embedding문서가 있으면 cache값을 읽어오고, 없으면 cache를 새로 만든다.
cached_embaddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embaddings)